In [3]:
import pandas as pd

In [4]:
root_path = 'data/root'
mortality_path = 'data/in-hospital-mortality'

In [5]:
if not os.path.exists(mortality_path):
    os.makedirs(mortality_path)


In [7]:
label_df = pd.read_csv(os.path.join('data/root/3', 'episode1.csv'))


In [8]:
label_df.iloc[0]["Mortality"]

0.0

In [14]:
ts_path = 'data/root/3/episode1_timeseries.csv'
with open(ts_path) as tsfile:
    ts_lines = tsfile.readlines()
    print(ts_lines)
    header = ts_lines[0]
    print()
    print(header)
    ts_lines = ts_lines[1:]
    print()
    print(ts_lines)
    for line in ts_lines:
        print(line)
        float(line.split(',')[0])

['Hours,Capillary refill rate,Diastolic blood pressure,Fraction inspired oxygen,Glascow coma scale eye opening,Glascow coma scale motor response,Glascow coma scale total,Glascow coma scale verbal response,Glucose,Heart Rate,Height,Mean blood pressure,Oxygen saturation,Respiratory rate,Systolic blood pressure,Temperature,Weight,pH\n', '16.666666666666668,,,,,,,,91.0,,,,,,,,,\n', '17.966666666666665,,,,,,,,,,,,,,,,,5.0\n', '18.0,,,,,,,,,,,,,,,,96.80000305175781,\n', '18.5,,,,,,,,,,,,,,,36.055599212646484,,\n', '18.75,,24.0,,,,,,,95,,36.666698455810547,98.0,16,62.0,,,\n', '19.0,,38.0,,,,,,,,,47.333301544189453,82.0,,66.0,,,\n', '19.2,,,,,,,,,,,,,,,,,7.35\n', '19.233333333333334,,,,,,,,140.0,,,,,,,,,7.59\n', '19.25,,38.0,,,,,,,,,97.666702270507812,74.0,,217.0,,,\n', '19.433333333333334,,,,,,,,162.0,,,,,,,,,\n', '19.5,,64.0,,,,,,,151,,76.666702270507812,,,102.0,,,\n', '19.75,,41.0,,,,,,,135,,58.666698455810547,,,94.0,,,\n', '19.983333333333334,,,,,,,,281.0,,,,,,,,,\n', '20.0,,49.0,,,,,,,143

In [ ]:

    ts_lines = ts_lines[1:]
    event_times = [float(line.split(',')[0]) for line in ts_lines]

    ts_lines = [line for (line, t) in zip(ts_lines, event_times)
                if -eps < t < n_hours + eps]

In [ ]:
def process_partition(root_path, mortality_path, partition, eps=1e-6, n_hours=48):
    mortality_dir = os.path.join(mortality_path, partition)
    if not os.path.exists(mortality_dir):
        os.mkdir(output_dir)

    xy_pairs = []
    patients = list(filter(str.isdigit, os.listdir(os.path.join(root_path, partition))))
    for patient in tqdm(patients, desc='Iterating over patients in {}'.format(partition)):
        # 得到某个病人的数据文件夹
        patient_folder = os.path.join(root_path, partition, patient)
        # 得到病人的episode*_timeseries.csv
        patient_ts_files = list(filter(lambda x: x.find("timeseries") != -1, os.listdir(patient_folder)))

        for ts_filename in patient_ts_files:
            with open(os.path.join(patient_folder, ts_filename)) as tsfile:
                lb_filename = ts_filename.replace("_timeseries", "")
                # 读取病人ICU和diagnose结合的信息 即episode*.csv
                label_df = pd.read_csv(os.path.join(patient_folder, lb_filename))

                # empty label file
                if label_df.shape[0] == 0:
                    continue
                # 得到该次病人入住ICU是否死亡的标志
                mortality = int(label_df.iloc[0]["Mortality"])
                if pd.isnull(los):
                    print("\n\t(length of stay is missing)", patient, ts_filename)
                    continue
                los = 24.0 * label_df.iloc[0]['Length of Stay']  # in hours
                # 排除掉病人入住ICU小于48h的数据
                if los < n_hours - eps:
                    continue
                # 读取了csv的全部内容
                ts_lines = tsfile.readlines()
                # header是列名
                header = ts_lines[0]
                ts_lines = ts_lines[1:]
                event_times = [float(line.split(',')[0]) for line in ts_lines]

                ts_lines = [line for (line, t) in zip(ts_lines, event_times)
                            if -eps < t < n_hours + eps]

                # no measurements in ICU
                if len(ts_lines) == 0:
                    print("\n\t(no events in ICU) ", patient, ts_filename)
                    continue

                output_ts_filename = patient + "_" + ts_filename
                with open(os.path.join(output_dir, output_ts_filename), "w") as outfile:
                    outfile.write(header)
                    for line in ts_lines:
                        outfile.write(line)

                xy_pairs.append((output_ts_filename, mortality))

    print("Number of created samples:", len(xy_pairs))
    if partition == "train":
        random.shuffle(xy_pairs)
    if partition == "test":
        xy_pairs = sorted(xy_pairs)

    with open(os.path.join(output_dir, "listfile.csv"), "w") as listfile:
        listfile.write('stay,y_true\n')
        for (x, y) in xy_pairs:
            listfile.write('{},{:d}\n'.format(x, y))
